In [5]:
# Import all other necessary modules
%load_ext autoreload
%autoreload 2
import cv2
import datetime
import io
import itertools
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import operator
import os
import poleno_db_interface.database.model.data_explorer_model as dem
import poleno_db_interface.database.model.poleno_data_model as pdm
from poleno_ml.database.query_interface_ml import QueryInterfaceML, DatasetPipeline
import random
import sklearn.metrics
from sqlalchemy import func
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras as keras
from tqdm.notebook import tqdm
from typing import List
from uuid import UUID
import uuid


# allow memory growth
for dev in tf.config.list_physical_devices():
    try:
        tf.config.experimental.set_memory_growth(
            dev, True
        )
        print(f"Success for {dev}")
    except:
        print(f"Failed for {dev}")

# specifies which PhysicalDevice objects are visible to the runtime. TF will only allocate memory and place operations on visible physical devices
gpu0 = tf.config.list_physical_devices('GPU')[0] # use GPU n
tf.config.set_visible_devices(gpu0, 'GPU')
tf.config.experimental.set_virtual_device_configuration(
    gpu0, 
    #[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=25_000)] # set max GPU memory usage
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=36_000)]
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Failed for PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
Failed for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Success for PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


In [19]:
import myloginpath
db_config = myloginpath.parse('client', path='/tf/.mylogin.cnf')

# Conect to the database and create an interface instance
query_interface_ml = QueryInterfaceML(**db_config)

In [6]:
# A dictionary that has the form of { <collection>: { <dataset-id>: <class-label> } }
# Collection name can be choosen freely and is intended to enable grouping of datasets into logical
# units independnet on class. Typically, this can be used to seperate datasets form different systems
# but can also be used to separate different years, sample source or any other propertie. Later in the
# notebook, you will then be able to create test and validation sets with different collections as
# source.
DATASET_DEFINITIONS = {
    
    "raw-pollens": {
        "11ea8493-7107-8db4-9bf7-ae7b87f820b4": "Alnus",
        "11ea847a-f995-790c-830f-ae7b87f820b4": "Alnus",
        "11ea8475-957e-347c-985a-ae7b87f820b4": "Alnus",
        "11ea8897-f50e-66a2-9876-ae7b87f820b4": "Betula",
        "11ea8632-18ed-7210-985a-ae7b87f820b4": "Betula",
        "11ea8632-1eb2-2452-bc84-ae7b87f820b4": "Betula",
        "11ea8f77-4ee3-aef4-b330-ae7b87f820b4": "Carpinus",
        "11ea8f6d-3e75-9fe6-b46e-ae7b87f820b4": "Carpinus",
        "11ea8f6d-1562-211a-8192-ae7b87f820b4": "Carpinus",
        "11ea8f6c-b78c-d076-a542-ae7b87f820b4": "Carpinus",
        "11ea8498-b729-d4e6-bc84-ae7b87f820b4": "Corylus",
        "11ea8498-b083-cb92-a1a5-ae7b87f820b4": "Corylus",
        "11ea8498-afa9-cec4-a877-ae7b87f820b4": "Corylus",
        "11ea8fa9-6c12-723a-b3dd-ae7b87f820b4": "Cupressus",
        "11ea8fa8-fafa-aeb4-ac46-ae7b87f820b4": "Cupressus",
        "11ea8fa8-d163-dce2-b1cb-ae7b87f820b4": "Cupressus",
        "11ea8636-313b-a6e4-a69e-ae7b87f820b4": "Fagus",
        "11ea8635-ef91-6ab2-a877-ae7b87f820b4": "Fagus",
        "11ea8635-eb18-6ee0-9876-ae7b87f820b4": "Fagus",
        "11ea857e-7bc5-60a0-842e-ae7b87f820b4": "Fraxinus",
        "11ea857b-3d52-9034-830f-ae7b87f820b4": "Fraxinus",
        "11ea857b-150e-c372-bc84-ae7b87f820b4": "Fraxinus",
        "11ea8af3-c533-f39e-8b25-ae7b87f820b4": "Pinaceae",
        "11ea8af1-91fc-9a46-8b25-ae7b87f820b4": "Pinaceae",
        "11ea8af0-83dc-6d66-b06c-ae7b87f820b4": "Pinaceae",
        "11ea863d-acf6-0ade-985a-ae7b87f820b4": "Pinaceae",
        "11ea863c-2449-be52-8814-ae7b87f820b4": "Pinaceae",
        "11ea8b83-25c9-8194-90d1-ae7b87f820b4": "Platanus",
        "11ea8881-3721-9aa8-a907-ae7b87f820b4": "Platanus",
        "11ea990f-ee01-8334-b3dd-ae7b87f820b4": "Poaceae",
        "11ea990c-b2bc-fe96-b46e-ae7b87f820b4": "Poaceae",
        "11eb5fd9-961a-313e-ac56-ae7b87f820b4": "Poaceae",
        "11eb5fd9-dd36-0a20-88f3-ae7b87f820b4": "Poaceae",
        "11ebe542-660e-0206-80be-ae7b87f820b4": "Poaceae",
        "11eb5fc3-03fa-6da2-8b42-ae7b87f820b4": "Poaceae",
        "11ebe540-187e-9a0c-b0e2-ae7b87f820b4": "Poaceae",
        "11ea8893-edfb-ca84-a877-ae7b87f820b4": "Populus",
        "11ea84a0-e89b-43b8-a69e-ae7b87f820b4": "Populus",
        "11ea84a0-a2f0-ab8c-a877-ae7b87f820b4": "Populus",
        "11ea863e-1fea-0f7c-a1a5-ae7b87f820b4": "Quercus",
        "11ea863e-1b86-8226-a1a5-ae7b87f820b4": "Quercus",
        "11ea863d-f388-a038-a1a5-ae7b87f820b4": "Quercus",
        "11ea8477-cede-e7dc-897d-ae7b87f820b4": "Taxus",
        "11ea8477-b584-b690-830f-ae7b87f820b4": "Taxus",
        "11ea8494-33a5-2e4e-bc84-ae7b87f820b4": "Taxus",
        "11ea849c-df8f-d95e-897d-ae7b87f820b4": "Ulmus",
        "11ea849c-db7b-2170-8b0f-ae7b87f820b4": "Ulmus",
        "11ea849a-0e25-4018-8814-ae7b87f820b4": "Ulmus",
    },
    
    "old-pollens": {
        "11ea5df1-de4e-68a2-bdea-ae7b87f820b4": "Alnus",
        "11ea5dec-aad2-40f2-adc5-ae7b87f820b4": "Alnus",
        "11ea5dea-76dd-45e6-9881-ae7b87f820b4": "Alnus",
        "11ea8318-8f19-8414-8f2c-ae7b87f820b4": "Betula",
        "11ea831d-c087-d5fa-8016-ae7b87f820b4": "Betula",
        "11ea8319-0f9b-2f84-8f2c-ae7b87f820b4": "Betula",
        "11ea74ee-1ae2-3f42-bdc8-ae7b87f820b4": "Carpinus",
        "11ea74ef-1f33-22e0-b530-ae7b87f820b4": "Carpinus",
        "11ea74ef-7256-f794-8624-ae7b87f820b4": "Carpinus",
        "11ea74ef-cf63-afc2-bdc8-ae7b87f820b4": "Carpinus",
        "11ea5e04-fc94-364e-81ab-ae7b87f820b4": "Corylus",
        "11ea5e04-ae5b-1ace-9881-ae7b87f820b4": "Corylus",
        "11ea5e00-93c5-6f88-81ab-ae7b87f820b4": "Corylus",
        "11ea74e8-0f90-f08a-9ea9-ae7b87f820b4": "Cupressus",
        "11ea74ea-1a2b-69ec-9846-ae7b87f820b4": "Cupressus",
        "11ea74ea-93f3-4862-bc81-ae7b87f820b4": "Cupressus",
        "11ea831e-9169-688c-9d84-ae7b87f820b4": "Fagus",
        "11ea831e-5779-4480-a7e8-ae7b87f820b4": "Fagus",
        "11ea831e-0698-1618-8016-ae7b87f820b4": "Fagus",
        "11ea8314-3a9a-8644-8fc4-ae7b87f820b4": "Fraxinus",
        "11ea8313-8fef-2358-8016-ae7b87f820b4": "Fraxinus",
        "11ea8313-1742-26b2-9d84-ae7b87f820b4": "Fraxinus",
        "11ea8af1-16b3-afbe-b419-ae7b87f820b4": "Pinaceae", # Picea
        "11ea8af1-8668-a68e-b06c-ae7b87f820b4": "Pinaceae", # Picea
        "11ea8af0-7bba-7a4c-9b82-ae7b87f820b4": "Pinaceae", # Picea
        "11ea863b-0dbc-a204-8b0f-ae7b87f820b4": "Pinaceae", # Pinus
        "11ea863c-0128-16ee-a69e-ae7b87f820b4": "Pinaceae", # Pinus
        "11ea831f-8774-33e2-b44c-ae7b87f820b4": "Plantanus",
        "11ea831f-3746-b084-a59a-ae7b87f820b4": "Plantanus",
        "11ea990d-99b7-329e-b330-ae7b87f820b4": "Poaceae",
        "11ea990c-a115-87c6-b46e-ae7b87f820b4": "Poaceae",
        "11ea9a91-09a7-100e-86f2-ae7b87f820b4": "Poaceae", # Cynosurus
        "11ea9a73-0d84-81ca-b3dd-ae7b87f820b4": "Poaceae", # Cynosurus
        "11ebde5f-23d5-5e8c-8d93-ae7b87f820b4": "Poaceae", # Dactylis
        "11ea9911-17a1-65b4-89a8-ae7b87f820b4": "Poaceae", # Dactylis
        "11ebde60-40fb-4aa2-a536-ae7b87f820b4": "Poaceae", # Trisetum
        "11ea830f-df57-8bdc-91a3-ae7b87f820b4": "Populus",
        "11ea74e3-b2e6-df38-b530-ae7b87f820b4": "Populus",
        "11ea74e4-5089-a266-8624-ae7b87f820b4": "Populus",
        "11ea863a-bdae-8df4-a69e-ae7b87f820b4": "Quercus",
        "11ea863a-2252-7b54-a1a5-ae7b87f820b4": "Quercus",
        "11ea8639-acca-ee52-aa3e-ae7b87f820b4": "Quercus",
        "11ea5deb-b5f0-b38e-9881-ae7b87f820b4": "Taxus",
        "11ea5deb-6fa4-5f7a-bd51-ae7b87f820b4": "Taxus",
        "11ea5df3-6b53-0252-adc5-ae7b87f820b4": "Taxus",
        "11ea74d0-dd4c-4d34-9ea9-ae7b87f820b4": "Ulmus",
        "11ea74cf-fc64-b108-8624-ae7b87f820b4": "Ulmus",
        "11ea5ef1-f146-eabe-ab02-ae7b87f820b4": "Ulmus",
    },

    #14 classes
    "new-pollens": { # same datasets as in "old-pollens" but cleaned using a different strategy
        "11ed3b18-1ba8-ee6a-a8d4-496190c661df": "Alnus",
        "11ed3821-9ab6-8dba-a8d4-496190c661df": "Alnus",
        "11ed382b-542a-0dea-a8d4-496190c661df": "Alnus",
        "11ed3832-b506-668e-a8d4-496190c661df": "Betula",
        "11ed3a58-07b4-deb6-a8d4-496190c661df": "Betula",
        "11ed38e1-b2f4-f996-a8d4-496190c661df": "Betula",
        "11ed5ea4-96ed-bdf8-acbd-a95f70cb44b0": "Carpinus",
        "11ed5f80-5dcc-9c24-acbd-a95f70cb44b0": "Carpinus",
        "11ed3a6b-5612-556c-a8d4-496190c661df": "Carpinus",
        "11ed3982-4837-efd6-a8d4-496190c661df": "Carpinus",
        "11ed38e0-6bee-47ec-a8d4-496190c661df": "Corylus",
        "11ed3a6f-f2ce-f140-a8d4-496190c661df": "Corylus",
        "11ed431c-682d-5986-a8d4-496190c661df": "Corylus",
        "11ed38ea-0973-6246-a8d4-496190c661df": "Cupressus",
        "11ed3a79-1b42-92c2-a8d4-496190c661df": "Cupressus",
        "11ed431e-c1e0-39ce-a8d4-496190c661df": "Cupressus",
        "11ed3a7c-3218-626c-a8d4-496190c661df": "Fagus",
        "11ed5f45-7fe1-e688-acbd-a95f70cb44b0": "Fagus",
        "11ed5f7b-c645-a4b2-acbd-a95f70cb44b0": "Fagus",
        "11ed3b22-8577-b13c-a8d4-496190c661df": "Fraxinus",
        "11ed5f4a-52c3-34a4-acbd-a95f70cb44b0": "Fraxinus",
        "11ed5f7e-c6f1-7f46-acbd-a95f70cb44b0": "Fraxinus",
        "11ed3d7f-b12e-ad76-a8d4-496190c661df": "Pinaceae", # Picea
        "11ed5f4d-49d0-5748-acbd-a95f70cb44b0": "Pinaceae", # Picea
        "11ed5f82-2199-44da-acbd-a95f70cb44b0": "Pinaceae", # Picea
        "11ed3d9b-7037-b6fc-a8d4-496190c661df": "Pinaceae", # Pinus
        "11ed5494-ad56-7e64-acbd-a95f70cb44b0": "Pinaceae", # Pinus
        "11ed5535-c71d-065a-acbd-a95f70cb44b0": "Pinaceae", # Pinus
        "11ed55ce-5aaf-a2ae-acbd-a95f70cb44b0": "Pinaceae", # Pinus
        "11ed55d0-cec8-bdae-acbd-a95f70cb44b0": "Pinaceae", # Pinus
        "11ed55d5-a393-64c2-acbd-a95f70cb44b0": "Pinaceae", # Pinus
        "11ed5907-4d09-f0b4-acbd-a95f70cb44b0": "Plantanus",
        "11ed6006-a52a-2448-acbd-a95f70cb44b0": "Plantanus",
        "11ed43d2-4043-b620-a8d4-496190c661df": "Poaceae",
        "11ed6009-3e1d-3184-acbd-a95f70cb44b0": "Poaceae",
        "11ed6030-3f50-655e-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed6035-488e-4564-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed6037-d194-cad4-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed6039-e772-8574-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed591a-064b-e868-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed591c-0732-bf5c-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed5922-9f70-fc56-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed5925-0949-93b6-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed5926-9845-1346-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed6032-1974-ee34-acbd-a95f70cb44b0": "Poaceae", # Dactylis
        "11ed6036-c877-51c0-acbd-a95f70cb44b0": "Poaceae", # Dactylis
        "11ed603b-8b16-bcf8-acbd-a95f70cb44b0": "Poaceae", # Dactylis
        "11ed6038-9eb8-c42a-acbd-a95f70cb44b0": "Poaceae", # Dactylis
        "11ed6010-ad76-a770-acbd-a95f70cb44b0": "Poaceae", # Dactylis
        "11ed4487-97ec-87e8-a8d4-496190c661df": "Poaceae", # Trisetum
        "11ed5928-4b40-315a-acbd-a95f70cb44b0": "Populus",
        "11ed6012-cffe-1c4a-acbd-a95f70cb44b0": "Populus",
        "11ed6041-5f45-1876-acbd-a95f70cb44b0": "Populus",
        "11ed5930-a266-322e-acbd-a95f70cb44b0": "Quercus",
        "11ed601d-4f24-ca46-acbd-a95f70cb44b0": "Quercus",
        "11ed6045-38bd-92ec-acbd-a95f70cb44b0": "Quercus",
        "11ed59c8-6d52-bbaa-acbd-a95f70cb44b0": "Taxus",
        "11ed602a-3932-49e0-acbd-a95f70cb44b0": "Taxus",
        "11ed604c-7275-f1b2-acbd-a95f70cb44b0": "Taxus",
        "11ed59ca-df2c-2188-acbd-a95f70cb44b0": "Ulmus",
        "11ed602d-6874-225c-acbd-a95f70cb44b0": "Ulmus",
        "11ed6048-9fb9-4b28-acbd-a95f70cb44b0": "Ulmus",
    },
    
    "other": {
        "11ed65b6-6a22-1968-b56b-ae7b87f820b4": "Artemisia",
        "11ed463e-f09f-6456-b550-ae7b87f820b4": "Cedrus",
        "11ec6179-fde0-042c-adac-ae7b87f820b4": "Iberulites",
        "11ec617a-fb8b-e3f2-80fb-ae7b87f820b4": "Iberulites",
        "11ec5821-b371-c3dc-8359-ae7b87f820b4": "Iberulites",
        "11ec5832-4fee-03b4-8561-ae7b87f820b4": "Iberulites",
        "11ebe542-f782-c172-bf10-ae7b87f820b4": "Waterdroplets",
        "11ebeabd-e224-d5c4-8b63-ae7b87f820b4": "Waterdroplets",
        "11ebedec-0da5-47ac-8066-ae7b87f820b4": "Waterdroplets",
        "11ebee15-1fea-4c68-9cd6-ae7b87f820b4": "Waterdroplets",
        "11eda79a-f000-b3a2-85ae-ae7b87f820b4": "Trash",
    },
    
    "spores": {
        "11ebf9db-f2e9-98cc-bc67-ae7b87f820b4": "Alternaria Solani",
        "11ec01b9-d571-ea8e-b7e1-ae7b87f820b4": "Fusarium Graminearum",
    },
}

In [7]:
model_name = 'prod_new+other_trash'

In [8]:
batch_size = 8 # the smaller, the more difficult the training becomes but it also generally means better generalization
epochs = 256 # max number of epochs (early stopping automatically interrupts the training if validation loss stops improving)
img_shape = (200,200,1)
model_features = [
     'rec0', 'rec1'  
] # 'rec0' and 'rec1' for holo images
data_filters = [
    #'blur', # remove blurry events
    #'crop', # remove cropped events
]
data_maps = [
    #'process_waves', # remove "waves" from all events
    #'holo_aug', # image augmentation
] # if you change the training data, you might want to apply the same transformations to the polenos as pre-processing steps
caching = True # whether or not cache the datasets locally for better performance

collections_train = [
    #"raw-pollens",
    #"old-pollens",
    "new-pollens",
    "other",
    #"spores",
]

# These two values only apply when collections_val is empty
train_part = 0.7
test_part = 0.3

# Leave empty if you'd like to train and eval on the same collections
collections_val = []

classes = set(cls 
              for collection in collections_train + collections_val 
              for cls in DATASET_DEFINITIONS[collection].values()) # not need to touch this

In [15]:
model_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [16]:
# model's info to save on disk
model_info = {
    'model_name': model_name,
    'model_timestamp': model_timestamp,
    'batch_size': batch_size,
    'model_features': model_features,
    'data_filters': data_filters,
    'data_maps': data_maps,
    'collections_train': collections_train,
    'train_part': train_part,
    'test_part': test_part,
    'collections_val': collections_val,
    'DATASET_DEFINITIONS': DATASET_DEFINITIONS,
}

In [17]:
def get_dataset_mapping(ds_map: dict, collections: List[str], classes: List[str]):
    """This method filters a dataset definition for specific systems and class labels
    and returns a flat dictionary with { <dataset-id>: <class-label> } """
    
    ret = {}
    for c in collections:
        ret.update(
            { key: value for key, value in ds_map[c].items() if value in classes}
        )
    return ret

def get_dataset_sizes(ds_map_flat: dict):
    """Return a dict with <dataset-id>: <class-size>"""
    
    dataset_sizes = {}
    for k, v in dataset_map.items():
        result = query_interface_ml.session.query(
            func.count(dem.EventsInEventDataset.event_id)
        ).filter(
            dem.EventsInEventDataset.dataset_id==uuid.UUID(k).bytes
        ).all()
        dataset_sizes[k] = result[0][0]
    return dataset_sizes
    
def get_class_sizes(ds_map_flat: dict, dataset_sizes: dict):
    """Return a dict with <class-name>: <class-size>"""
    
    class_sizes = {}
    for k, v in dataset_map.items():
        size = dataset_sizes[k]
        if v not in class_sizes: class_sizes[v] = 0
        class_sizes[v] += size
    return class_sizes

def get_sorted_class_list(ds_map_flat: dict):
    return sorted(list(set(ds_map_flat.values())))

In [20]:
# Get the dataset collection you need for the training
dataset_map = get_dataset_mapping(
    DATASET_DEFINITIONS,
    collections=collections_train,
    classes=classes
)
assert len(dataset_map) > 0

dataset_sizes = get_dataset_sizes(dataset_map)
class_sizes = get_class_sizes(dataset_map, dataset_sizes)
classes = get_sorted_class_list(dataset_map)
num_classes = len(classes)

n_samples = sum(dataset_sizes.values())
class_counts = [class_sizes[d] for d in classes]
class_weights = n_samples / np.array(class_counts)
model_info['classes'] = classes
model_info['class_weights'] = list(class_weights)

In [21]:
if len(collections_val) == 0:

    dataset_train, dataset_val = query_interface_ml.prepare_tf_dataset_from_poleno_datasets(
        dataset_list=list(dataset_map.keys()),
        batch_size=batch_size,
        model_features=model_features,
        labels=classes,
        dataset_label_mapping=dataset_map,
        split=(train_part, test_part)
    )

else:
    
    dataset_map_val = get_dataset_mapping(
        DATASET_DEFINITIONS,
        collections=collections_val,
        classes=classes
    )
    
    dataset_train = query_interface_ml.prepare_tf_dataset_from_poleno_datasets(
        dataset_list=list(dataset_map.keys()),
        batch_size=batch_size,
        model_features=model_features,
        labels=classes,
        dataset_label_mapping=dataset_map,
    )
    
    dataset_val = query_interface_ml.prepare_tf_dataset_from_poleno_datasets(
        dataset_list=list(dataset_map_val.keys()),
        batch_size=batch_size,
        model_features=model_features,
        labels=classes,
        dataset_label_mapping=dataset_map_val,
    )
    

In [4]:
#num_classes = 18

In [22]:
inputs = []
paths = []

if "rec0" in model_features and "rec1" in model_features:
    input0 = keras.layers.Input(shape=[200,200,1], name="rec0")
    inputs.append(input0)
    input1 = keras.layers.Input(shape=[200,200,1], name="rec1")
    inputs.append(input1)

    ###                                   -> VVVV <- Change here to B0 - B7 if needed.
    path0 = keras.layers.Conv2D(64, (5,5), padding='same', activation='relu')(input0)
    path0 = keras.layers.Conv2D(64, (5,5), padding='same', activation='relu')(path0)
    path0 = keras.layers.MaxPool2D(2, strides=(2,2),padding='same')(path0)
    path0 = keras.layers.Dropout(0.2)(path0)
    path0 = keras.layers.Conv2D(64, (3,3), padding='same', activation='relu')(path0)
    path0 = keras.layers.Conv2D(64, (3,3), padding='same', activation='relu')(path0)
    path0 = keras.layers.MaxPool2D(2, strides=(2,2),padding='same')(path0)
    path0 = keras.layers.Dropout(0.2)(path0)
    path0 = keras.layers.Conv2D(128, (3,3), padding='same', activation='relu')(path0)
    path0 = keras.layers.Conv2D(128, (3,3), padding='same', activation='relu')(path0)
    path0 = keras.layers.Conv2D(128, (3,3), padding='same', activation='relu')(path0)
    path0 = keras.layers.MaxPool2D((2,2), strides=(2,2),padding='same')(path0)
    path0 = keras.layers.Dropout(0.2)(path0)
    path0 = keras.layers.Conv2D(256, (3,3), padding='same', activation='relu')(path0)
    path0 = keras.layers.Conv2D(256, (3,3), padding='same', activation='relu')(path0)
    path0 = keras.layers.Conv2D(256, (3,3), padding='same', activation='relu')(path0)
    path0 = keras.layers.MaxPool2D((2,2), strides=(2,2),padding='same')(path0)
    path0 = keras.layers.Dropout(0.2)(path0)

    path1 = keras.layers.Conv2D(64, (5,5), padding='same', activation='relu')(input1)
    path1 = keras.layers.Conv2D(64, (5,5), padding='same', activation='relu')(path1)
    path1 = keras.layers.MaxPool2D(2, strides=(2,2),padding='same')(path1)
    path1 = keras.layers.Dropout(0.2)(path1)
    path1 = keras.layers.Conv2D(64, (3,3), padding='same', activation='relu')(path1)
    path1 = keras.layers.Conv2D(64, (3,3), padding='same', activation='relu')(path1)
    path1 = keras.layers.MaxPool2D(2, strides=(2,2),padding='same')(path1)
    path1 = keras.layers.Dropout(0.2)(path1)
    path1 = keras.layers.Conv2D(128, (3,3), padding='same', activation='relu')(path1)
    path1 = keras.layers.Conv2D(128, (3,3), padding='same', activation='relu')(path1)
    path1 = keras.layers.Conv2D(128, (3,3), padding='same', activation='relu')(path1)
    path1 = keras.layers.MaxPool2D((2,2), strides=(2,2),padding='same')(path1)
    path1 = keras.layers.Dropout(0.2)(path1)
    path1 = keras.layers.Conv2D(256, (3,3), padding='same', activation='relu')(path1)
    path1 = keras.layers.Conv2D(256, (3,3), padding='same', activation='relu')(path1)
    path1 = keras.layers.Conv2D(256, (3,3), padding='same', activation='relu')(path1)
    path1 = keras.layers.MaxPool2D((2,2), strides=(2,2),padding='same')(path1)
    path1 = keras.layers.Dropout(0.2)(path1)

    ###
    
    holo_path_ = keras.layers.Concatenate()([path0, path1])
    holo_path_ = keras.layers.Flatten()(holo_path_)
    
    paths.append(holo_path_)

if "fl_spectra" in model_features:
    input_fl = keras.layers.Input(shape=[13], name="fl_spectra")
    fl_path = input_fl * 255
    fl_path = keras.layers.Dense(255)(fl_path)
    inputs.append(input_fl)
    paths.append(fl_path)

if len(paths) > 1:
    path_ = keras.layers.Concatenate()(paths)
else:
    path_ = paths[0]

#path_ = keras.layers.Dropout(.2)(path_)
outputs = keras.layers.Dense(
    num_classes,
    activation="softmax",
    name="target"
)(path_)
#outputs = keras.layers.Softmax(name="target")(outputs)


model = keras.Model(inputs=inputs, outputs=[outputs])

"done"

'done'

In [23]:
# To compile the model, we need a special loss funtion that allows for class weights. The details are not that important. Relavent to know
# is that this allows for inbalanced data to be trained correctly. For example, if you have two times more of class 1 than in class 2, then
# you would like the model to ignore this difference and act as if the two sets are identical in size.

class WeightedSCCE(tf.keras.losses.Loss):
    """Custom SparseCategoricalCrossentropy loss class that supports class weights."""
    def __init__(self, class_weight, from_logits=False, name='weighted_scce'):
        if class_weight is None or all(v == 1. for v in class_weight):
            self.class_weight = None
        else:
            self.class_weight = tf.convert_to_tensor(class_weight,
                dtype=tf.float32)
        self.reduction = keras.losses.Reduction.NONE
        self.unreduced_scce = keras.losses.SparseCategoricalCrossentropy(
            from_logits=from_logits, name=name,
            reduction=self.reduction)
        self.name = name

    def __call__(self, y_true, y_pred, sample_weight=None):
        loss = self.unreduced_scce(y_true, y_pred, sample_weight)
        if self.class_weight is not None:
            weight_mask = tf.gather(self.class_weight, y_true)
            loss = tf.math.multiply(loss, weight_mask)
        return loss

In [ ]:
# Finally we compile the ml model
learning_rate = 0.000_005
model.compile(
    # Optimizer, that handles the weight adjustment while training
    optimizer=keras.optimizers.Adam(learning_rate),  
    # Loss function to minimize
    loss=WeightedSCCE(class_weights),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [34]:
model_path = 'models'
model_file_path = os.path.join(model_path, model_name, 'model')
model_info_file_path = os.path.join(model_path, model_name, 'model', 'model_info.json')
checkpoint_file_path = os.path.join(model_path, model_name, 'training', 'checkpoints', '20230209-075841')

In [35]:
# reload last checkpoint's weights those are the ones to export
model.load_weights(checkpoint_file_path)
# save the best model
model.save(model_file_path)
# save the model's essential info
#with open(model_info_file_path, 'w') as f:
 #   f.write(json.dumps(model_info))

INFO:tensorflow:Assets written to: models/prod_new+other_trash/model/assets


INFO:tensorflow:Assets written to: models/prod_new+other_trash/model/assets


In [31]:
checkpoint_file_path

'models/prod_new+other_trash/training/checkpoints/20230207-163413'

In [33]:
os.listdir('models/prod_new+other_trash/training/checkpoints/')

['checkpoint', '20230209-075841.index', '20230209-075841.data-00000-of-00001']